In [1]:
import torch
import re
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
from model import AlexNet
from torchvision import models
from tqdm.notebook import tqdm

## Create Iterable dataset

I think we can just hard code the normalization mean and std.<br>
The point of normalizing is to help with training.<br>
Just make sure that we are using the same normalization for all datasets (train, valid, test).<br>

**I realize that we don't need to get the training mean and std because pixel values have a fixed range -- we can just scale using 225.**

In [2]:
class _Dataset(torch.utils.data.IterableDataset):    
    def __init__(self, trainval_list, indices, data_dir, transforms = None, label_mapping = dict()):
        super().__init__()
        self.data_dir = data_dir
        self.trainval_list = trainval_list
        self.list_ = [self.trainval_list[i] for i in indices]
        self.transforms = transforms
        self.label_mapping = label_mapping
        
    def __len__(self):
        return len(self.list_)
        
    def __iter__(self):
        for fn in self.list_:
            with Image.open(f"{self.data_dir}/The Oxford-IIIT Pet Dataset/images/{fn}.jpg") as img:
                data = np.asarray(img)
            if data.shape[-1] != 3:
                data = data[:,:,:3]
            if self.transforms:
                data = self.transforms(data)
            label = re.findall(r"^(.*)_[0-9]+", fn)
            label = label[0] if label else ""
            if label in self.label_mapping:
                label = self.label_mapping[label]
            else:
                label = len(self.label_mapping)
            yield data, label

def _create_label_mapping(trainval_list):
    cats = [re.findall(r"^(.*)_[0-9]+", i)[0] for i in trainval_list]
    temp = dict(enumerate(set(cats)))
    return {temp[i]: i for i in temp}
    
def PetsDataset(data_dir = "../data/", transforms = None):
    with open(data_dir+"The Oxford-IIIT Pet Dataset/annotations/trainval.txt") as f:
        trainval_txt = f.read()
    trainval_list = [f"{line.split()[0]}" for line in  trainval_txt.split("\n") if len(line.split())>1]
    rand_perm = np.random.permutation(len(trainval_list))
    train_idx = rand_perm[:int(len(trainval_list)*0.9)]
    valid_idx = rand_perm[int(len(trainval_list)*0.9):]
    
    mapping = _create_label_mapping(trainval_list)
    
    train_dataset = _Dataset(trainval_list, train_idx, data_dir, transforms, mapping)
    valid_dataset = _Dataset(trainval_list, valid_idx, data_dir, transforms, mapping)
    
    return train_dataset, valid_dataset

## Create transforms

In [3]:
T = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation((-15,15)),
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ColorJitter(),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

## Create Datasets

In [4]:
train_ds, valid_ds = PetsDataset(transforms = T)

## Create DataLoader

In [5]:
dataloader = torch.utils.data.DataLoader(train_ds, batch_size = 32)

In [6]:
valid_dataloader = torch.utils.data.DataLoader(valid_ds, batch_size = 100)

In [7]:
foo = next(iter(dataloader))

In [8]:
foo[1]

tensor([15, 22, 26, 25,  2, 30, 15, 29, 18,  7, 10, 10, 33, 17, 15, 26, 14,  8,
        29, 33, 11, 10,  7, 22,  1,  2, 14, 11, 11, 31, 12, 22])

## Train our model

In [13]:
class OurModel (torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.alexnet = AlexNet()
        self.linear1 = torch.nn.Linear(1000,500)
        self.linear2 = torch.nn.Linear(500,250)
        self.linear3 = torch.nn.Linear(250, 100)
        self.linear4 = torch.nn.Linear(100,38)
        
    def forward(self, x):
        x = torch.nn.functional.relu(self.alexnet(x))
        x = torch.nn.functional.relu(self.linear1(x))
        x = torch.nn.functional.relu(self.linear2(x))
        x = torch.nn.functional.relu(self.linear3(x))
        x = self.linear4(x)
        return x


In [14]:
model = OurModel()
model.cuda()

OurModel(
  (alexnet): AlexNet(
    (max_pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (alexnet1a): _AlexNet1(
      (conv1): Conv2d(3, 48, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (conv2): Conv2d(48, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (res_norm): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
      (max_pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (alexnet1b): _AlexNet1(
      (conv1): Conv2d(3, 48, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (conv2): Conv2d(48, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (res_norm): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
      (max_pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (alexnet2a): _AlexNet2(
      (res_norm): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
      (max_pool): MaxPool2d(kernel_size=3, stride=2, padding

In [15]:
loss = torch.nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters())

In [16]:
def get_accuracy(model, valid_dataloader):
    accumulated_accuracy = []
    for batch, truth in valid_dataloader:
        batch = batch.to("cuda")
        truth = truth.to("cuda")
        weighted_rate = (model(batch).argmax(dim = 1) == truth).sum().cpu().numpy()
        accumulated_accuracy.append(weighted_rate)
    return sum(accumulated_accuracy)/len(valid_dataloader)

In [17]:
get_accuracy(model, valid_dataloader)

0.019021739130434784

In [18]:
epochs = 10

train_loss = []

for epoch in range(epochs):
    print(f"---Epoch {epoch}---")
    print(f"Accuracy: {get_accuracy(model, valid_dataloader)}")
    for batch, preds in dataloader:
        batch = batch.to("cuda")
        preds = preds.to("cuda")
        y_pred = model(batch)
        
        l = loss(y_pred, preds)
        train_loss.append(l)
          
        l.backward()
           
        optim.step()
        optim.zero_grad()
    print(f"Training Loss: {train_loss[-1]}")
    print("-"*30)

---Epoch 0---
Accuracy: 0.019021739130434784
Training Loss: 3.650052309036255
------------------------------
---Epoch 1---
Accuracy: 0.021739130434782608
Training Loss: 3.6440470218658447
------------------------------
---Epoch 2---
Accuracy: 0.021739130434782608
Training Loss: 3.642866373062134
------------------------------
---Epoch 3---
Accuracy: 0.021739130434782608


KeyboardInterrupt: 

## Let's try training with Transfer Learning

In [12]:
class OurModel (torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = models.resnet18(pretrained = True)
        self.last_layers = torch.nn.Sequential(
                torch.nn.Linear(1000,512),
                torch.nn.ReLU(),
                torch.nn.Linear(512,256),
                torch.nn.ReLU(),
                torch.nn.Linear(256,38)
        )
        
        #1000,38
        
    def forward(self, x):
        x = torch.nn.functional.relu(self.resnet(x))
        x = self.last_layers(x)
        return x


In [13]:
model = OurModel()
model.cuda()

OurModel(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_run

In [17]:
loss = torch.nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.last_layers.parameters())

In [18]:
get_accuracy(model, valid_dataloader)

0.02717391304347826

In [19]:
epochs = 10

train_loss = []

for epoch in range(epochs):
    print(f"---Epoch {epoch}---")
    print(f"Accuracy: {get_accuracy(model, valid_dataloader)}")
    for batch, preds in dataloader:
        batch = batch.to("cuda")
        preds = preds.to("cuda")
        y_pred = model(batch)
        
        l = loss(y_pred, preds)
        train_loss.append(l)
          
        l.backward()
           
        optim.step()
        optim.zero_grad()
    print(f"Training Loss: {train_loss[-1]}")
    print("-"*30)

---Epoch 0---
Accuracy: 0.02717391304347826
Training Loss: 1.4094126224517822
------------------------------
---Epoch 1---
Accuracy: 0.8043478260869565
Training Loss: 0.4521753787994385
------------------------------
---Epoch 2---
Accuracy: 0.8478260869565217
Training Loss: 0.5485384464263916
------------------------------
---Epoch 3---
Accuracy: 0.8641304347826086
Training Loss: 0.18103063106536865
------------------------------
---Epoch 4---
Accuracy: 0.8125
Training Loss: 0.3133333921432495
------------------------------
---Epoch 5---
Accuracy: 0.8260869565217391
Training Loss: 0.22781527042388916
------------------------------
---Epoch 6---
Accuracy: 0.8478260869565217
Training Loss: 0.10428817570209503
------------------------------
---Epoch 7---
Accuracy: 0.8505434782608695
Training Loss: 0.5771973133087158
------------------------------
---Epoch 8---
Accuracy: 0.845108695652174
Training Loss: 0.12190097570419312
------------------------------
---Epoch 9---
Accuracy: 0.8532608695

In [20]:
get_accuracy(model, valid_dataloader)

0.842391304347826